# Observational study 

In [17]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotnine
from networkx import nx
from plotnine import ggplot, aes, geom_bar
from functions import *
import spacy
import time
import datetime
import statsmodels.formula.api as smf

In [2]:
from popularity import trending_history
from financials import *

In [3]:
elon_org_df = pd.read_csv('Data/org-lg-Elon Musk.csv.bz2')

## Matching : 

In [4]:
def compareCompanies(companies, tickers, date, shares, hist):
    # Returns a df with the following columns: ['Date', 'Close', 'Volume', 'MarketCap', 'Popularity']
    df = pd.DataFrame(companies, columns=['Name'])
    df['Date'] = date
    
    marketData = []

    for i in range(len(tickers)):
        marketData.append(obs_info(tickers[i], date, shares[i], hist[i]).values.tolist())
    
    df['Close'] = [i[0] for i in marketData]
    df['Volume'] = [i[1] for i in marketData]
    df['MarketCap'] = [i[2] for i in marketData]
    
    df_trend = trending_history(companies, date)
    df_trend_list = df_trend.values.tolist()[0]
    df['Popularity'] = df_trend_list
    
    return df

def obs_info(ticker, date, sharesOutstanding, comp_hist):
    for i in range(len(comp_hist)):
        if comp_hist.iloc[i].name.date() > datetime.date.fromisoformat(date):
            df = comp_hist.iloc[i]
            break

    df['MarketCap'] = sharesOutstanding*df['Close']
    
    return df

In [5]:
#Data is our data frame with all the companies with first column 1 or 0 if Elon Musk talks about it.
# after,Name of the company and the other four covariates 
def compare(company_quote, date, companies, tickers, shares, hist): 

    company = compareCompanies(companies, tickers, date, shares, hist)

    company ['Elon']= 0
    company ['compare'] = company_quote
    company.at[4,'Elon'] = 1
    company.at[4,'compare'] = 'None'
    return company
    
#Call this function for all our functions and a set of company that will represent the controled one.

In [18]:
def create_data(companies_list):
    Company = companies_list
    company_quotes = Company [-1]
    quotes = elon_org_df[elon_org_df['ORG'] == company_quotes]
    first_date = quotes.iloc[0]['date'][0:10]
    ticker =[ticker_of_company(Company[0]),
             ticker_of_company(Company[1]),
             ticker_of_company(Company[2]),
             ticker_of_company(Company[3]),
             ticker_of_company(Company[4])] 
    shares=[yf.Ticker(ticker[0]).info['sharesOutstanding'],
            yf.Ticker(ticker[1]).info['sharesOutstanding'],
            yf.Ticker(ticker[2]).info['sharesOutstanding'],
            yf.Ticker(ticker[3]).info['sharesOutstanding'],
            yf.Ticker(ticker[4]).info['sharesOutstanding']]
    hist =[yf.Ticker(ticker[0]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[1]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[2]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[3]).history(start=first_date)[['Close', 'Volume']],
           yf.Ticker(ticker[4]).history(start=first_date)[['Close', 'Volume']]]
    data = pd.DataFrame(columns = ['Name','Date','Close','Volume','MarketCap','Popularity','Elon','compare'])
    for i in range (len(quotes)):
        time.sleep(1)
        date = quotes.iloc[i,:]['date'][0:10]
        company = compare(company_quotes, date, Company, ticker, shares, hist)
        frame = [data, company]
        data = pd.concat(frame)
    return data

In [19]:
#Apple: 
data_Apple = create_data(['Microsoft','IBM','Samsung','Dell','Apple'] )

KeyboardInterrupt: 

In [15]:
#PayPal : 
data_PayPal = create_data(['Western Union','EuroNet','MoneyGram','Payoneer','PayPal'])

<ipython-input-4-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
#Twitter : 
data_Twitter = create_data(['Google','Facebook','Snapchat','Pinterest','Twitter'])

<ipython-input-4-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
#Tesla: 
data_Tesla = create_data(['Rivian','Lucid','VolksWagen','General Motors','Tesla'])

<ipython-input-4-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


ResponseError: The request failed: Google returned a response with code 429.

In [11]:
data_Tesla

NameError: name 'data_Tesla' is not defined

In [ ]:
for i in range(100,2200,100): 
    data_t = create_data(['Rivian','Lucid','VolksWagen','General Motors','Tesla'],i,i+100)
    frame = [ data_Tesla, data_t]
    data_Tesla = pd.concat(frame)
    

In [17]:
#Ford : 
data_Ford = create_data(['Toyota','BMW','General Motors','Stellantis','Ford'])

<ipython-input-4-fb78ad33690e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_Tesla

In [22]:
frame = [data_Apple,data_Twitter, data_Ford, data_PayPal]
data = pd.concat(frame)
data.to_csv('Data/FinalFeatures.csv.bz2' , index=False)

In [6]:
data = pd.read_csv('Data/FinalFeatures.csv.bz2')
data

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare
0,Microsoft,2015-10-09,42.247707,19769100.0,3.171949e+11,30,0,Apple
1,IBM,2015-10-09,109.634865,3376070.0,9.826792e+10,3,0,Apple
2,Samsung,2015-10-09,21274.496094,18948700.0,1.270041e+14,100,0,Apple
3,Dell,2015-10-09,12.067676,271519.0,3.499397e+09,16,0,Apple
4,Apple,2015-10-09,25.580677,121868800.0,4.196868e+11,56,1,None
...,...,...,...,...,...,...,...,...
1195,Western Union,2020-01-21,25.709953,3686900.0,1.033558e+10,13,0,PayPal
1196,EuroNet,2020-01-21,167.250000,304800.0,8.840166e+09,0,0,PayPal
1197,MoneyGram,2020-01-21,2.680000,2613100.0,2.457249e+08,4,0,PayPal
1198,Payoneer,2020-01-21,9.680000,1151400.0,3.281598e+09,1,0,PayPal


In [7]:
#Propensity score : 
def add_propensityscore (data):#so that if we want to change only the first line don't have to do it
    #With a column of one and zero if elon musk talked about you 
    #company = company_quote.extend(controled)
    #data = compareCompanies(company,date)
    #Normalise the features 
    data['Close'] = (data['Close'] - data['Close'].mean())/data['Close'].std()
    data['Volume']= (data['Volume']-data['Volume'].mean())/data['Volume'].std()
    data['MarketCap']= (data['MarketCap']-data['MarketCap'].mean())/data['MarketCap'].std()
    data['Popularity']= (data['Popularity']-data['Popularity'].mean())/data['Popularity'].std()
    #Create the model
    mod = smf.logit(formula='Elon ~ Close+ Volume + MarketCap + Popularity', data=data)
    res = mod.fit()
    data['Propensity_score'] = res.predict()

In [8]:
add_propensityscore(data)

Optimization terminated successfully.
         Current function value: 0.223971
         Iterations 15


/Users/selimajaoua/opt/anaconda3/lib/python3.8/site-packages/statsmodels/discrete/discrete_model.py:1810: RuntimeWarning: overflow encountered in exp


In [13]:
data

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Propensity_score
0,Microsoft,2015-10-09,-0.286007,-0.147748,-0.272609,-0.255595,0,Apple,0.129603
1,IBM,2015-10-09,-0.277090,-0.474358,-0.277450,-0.940792,0,Apple,0.001894
2,Samsung,2015-10-09,2.523743,-0.164094,2.528635,1.520841,0,Apple,0.000000
3,Dell,2015-10-09,-0.290001,-0.536212,-0.279545,-0.610883,0,Apple,0.126851
4,Apple,2015-10-09,-0.288213,1.886453,-0.270343,0.404224,1,None,0.939514
...,...,...,...,...,...,...,...,...,...
1195,Western Union,2020-01-21,-0.288196,-0.468165,-0.279394,-0.687016,0,PayPal,0.074953
1196,EuroNet,2020-01-21,-0.269465,-0.535549,-0.279427,-1.016925,0,PayPal,0.000162
1197,MoneyGram,2020-01-21,-0.291244,-0.489559,-0.279617,-0.915415,0,PayPal,0.110559
1198,Payoneer,2020-01-21,-0.290317,-0.518681,-0.279550,-0.991548,0,PayPal,0.072975


In [9]:
def get_similarity(propensity_score1, propensity_score2):
    '''Calculate similarity for instances with given propensity scores'''
    return 1-np.abs(propensity_score1-propensity_score2)

In [51]:
def matchingfunction(companies):
    treatment_df = data[data['Elon'] == 1]
    control_df   = data[data['Elon'] == 0]
    # Create an empty undirected graph
    G = nx.Graph()
    # Loop through all the pairs of instances
    for control_id, control_row in control_df.iterrows():
        for treatment_id, treatment_row in treatment_df.iterrows():
            if (control_row['compare']== treatment_row['Name']) and (control_row['Date']== treatment_row['Date']) :
                # Calculate the similarity 
                similarity = get_similarity(control_row['Propensity_score'],
                                    treatment_row['Propensity_score'])
                # Add an edge between the two instances weighted by the similarity between them
                G.add_weighted_edges_from([(control_id, treatment_id, similarity)])
    # Generate and return the maximum weight matching on the generated graph
    matching = nx.max_weight_matching(G)
    return matching

In [54]:
matching = matchingfunction(data)
matched = [i[0] for i in list(matching)]+ [i[1]for i in list(matching)]
balanced = data.iloc[matched]


## After matching : 


In [55]:
balanced

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Propensity_score
474,Twitter,2017-03-02,-0.289514,-0.238891,-0.279344,-0.864659,1,None,0.103594
556,Facebook,2018-10-05,-0.270789,-0.062537,-0.271395,1.520841,0,Twitter,0.054540
482,Snapchat,2017-06-12,-0.289203,-0.304546,-0.279080,-0.991548,0,Twitter,0.070382
1034,Ford,2019-03-15,-0.290540,0.199505,-0.278928,1.520841,1,None,0.964482
1089,Ford,2019-07-01,-0.290328,0.050271,-0.278790,1.520841,1,None,0.954280
...,...,...,...,...,...,...,...,...,...
686,Facebook,2019-02-20,-0.270420,-0.230657,-0.271249,1.520841,0,Twitter,0.039636
115,Microsoft,2015-10-09,-0.286007,-0.147748,-0.272609,-0.255595,0,Apple,0.129603
754,Twitter,2020-01-31,-0.287222,-0.258887,-0.279038,-0.585505,1,None,0.089250
623,Pinterest,2018-07-26,-0.288369,1.193920,-0.279319,-0.966170,0,Twitter,0.318925


In [35]:
matching (data)

TypeError: 'set' object is not callable

In [41]:
treated = balanced.loc[balanced['Elon']==1]

In [42]:
controled = balanced.loc[balanced['Elon']==0]

In [43]:
treated

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Propensity_score
474,Twitter,2017-03-02,-0.289514,-0.238891,-0.279344,-0.864659,1,None,0.103594
1034,Ford,2019-03-15,-0.290540,0.199505,-0.278928,1.520841,1,None,0.964482
1089,Ford,2019-07-01,-0.290328,0.050271,-0.278790,1.520841,1,None,0.954280
784,Twitter,2020-03-03,-0.286834,-0.243159,-0.278986,-0.610883,1,None,0.078418
504,Twitter,2018-07-13,-0.285741,-0.282359,-0.278840,-0.788526,1,None,0.040009
...,...,...,...,...,...,...,...,...,...
509,Twitter,2018-07-18,-0.285850,-0.275306,-0.278855,-0.788526,1,None,0.041627
439,Apple,2018-08-11,-0.284913,1.521746,-0.261297,0.201203,1,None,0.707946
79,Apple,2015-08-20,-0.288277,4.397824,-0.270517,0.125070,1,None,0.996348
754,Twitter,2020-01-31,-0.287222,-0.258887,-0.279038,-0.585505,1,None,0.089250


In [44]:
controled 

,Name,Date,Close,Volume,MarketCap,Popularity,Elon,compare,Propensity_score
556,Facebook,2018-10-05,-0.270789,-0.062537,-0.271395,1.520841,0,Twitter,0.054540
482,Snapchat,2017-06-12,-0.289203,-0.304546,-0.279080,-0.991548,0,Twitter,0.070382
1155,Western Union,2017-11-01,-0.289322,-0.446498,-0.279470,-0.712393,0,PayPal,0.099560
762,Snapchat,2020-01-31,-0.289186,0.065329,-0.279076,-0.991548,0,Twitter,0.110286
1011,BMW,2019-07-22,-0.283302,-0.471280,-0.278788,0.861022,0,Ford,0.271211
...,...,...,...,...,...,...,...,...,...
901,BMW,2018-11-05,-0.282857,-0.514572,-0.278743,0.810267,0,Ford,0.218455
1165,Western Union,2018-05-05,-0.289314,-0.407107,-0.279469,-0.509372,0,PayPal,0.146472
686,Facebook,2019-02-20,-0.270420,-0.230657,-0.271249,1.520841,0,Twitter,0.039636
115,Microsoft,2015-10-09,-0.286007,-0.147748,-0.272609,-0.255595,0,Apple,0.129603


In [56]:
matching


{(0, 4),
 (10, 14),
 (18, 19),
 (24, 20),
 (28, 29),
 (35, 284),
 (40, 269),
 (49, 45),
 (58, 59),
 (64, 60),
 (68, 54),
 (69, 53),
 (74, 70),
 (75, 79),
 (89, 245),
 (90, 244),
 (99, 95),
 (100, 239),
 (104, 235),
 (105, 229),
 (110, 199),
 (114, 195),
 (120, 219),
 (124, 215),
 (129, 125),
 (130, 189),
 (144, 270),
 (145, 139),
 (149, 135),
 (154, 150),
 (155, 159),
 (160, 164),
 (169, 165),
 (170, 174),
 (184, 180),
 (185, 134),
 (190, 119),
 (194, 115),
 (200, 179),
 (204, 175),
 (208, 209),
 (210, 214),
 (224, 220),
 (225, 109),
 (234, 255),
 (240, 94),
 (249, 85),
 (250, 254),
 (259, 230),
 (264, 260),
 (265, 44),
 (274, 140),
 (275, 279),
 (280, 39),
 (289, 285),
 (290, 34),
 (294, 30),
 (299, 295),
 (300, 84),
 (304, 80),
 (305, 9),
 (309, 5),
 (313, 314),
 (318, 319),
 (320, 324),
 (329, 325),
 (330, 334),
 (339, 335),
 (344, 343),
 (348, 349),
 (353, 354),
 (358, 359),
 (368, 364),
 (369, 363),
 (373, 374),
 (378, 379),
 (384, 383),
 (388, 389),
 (393, 394),
 (408, 409),
 (41

In [57]:
matched

[474,
 556,
 482,
 1034,
 1089,
 784,
 1155,
 762,
 1011,
 956,
 683,
 1076,
 504,
 200,
 210,
 1139,
 368,
 294,
 694,
 184,
 408,
 801,
 353,
 811,
 18,
 304,
 40,
 778,
 779,
 1091,
 1064,
 468,
 1130,
 849,
 934,
 891,
 596,
 99,
 318,
 1192,
 35,
 384,
 496,
 609,
 225,
 149,
 1056,
 718,
 969,
 1179,
 114,
 489,
 448,
 69,
 224,
 443,
 841,
 289,
 642,
 996,
 769,
 914,
 626,
 68,
 74,
 954,
 129,
 712,
 824,
 961,
 1170,
 1160,
 1081,
 1026,
 160,
 633,
 971,
 330,
 1144,
 275,
 190,
 1154,
 612,
 413,
 358,
 547,
 816,
 264,
 531,
 944,
 1096,
 104,
 1009,
 1074,
 529,
 876,
 677,
 259,
 309,
 185,
 936,
 794,
 899,
 1114,
 601,
 208,
 49,
 169,
 1197,
 544,
 124,
 1001,
 265,
 1066,
 673,
 728,
 723,
 89,
 105,
 250,
 300,
 1129,
 649,
 388,
 424,
 736,
 592,
 704,
 454,
 1105,
 657,
 906,
 453,
 299,
 516,
 571,
 889,
 636,
 759,
 155,
 1049,
 305,
 145,
 916,
 523,
 1184,
 976,
 274,
 1041,
 834,
 696,
 554,
 120,
 617,
 0,
 10,
 28,
 708,
 1024,
 428,
 204,
 986,
 881,
 114

In [ ]:
before =time.time()
quotes_Apple = elon_org_df[elon_org_df['ORG'] == 'Apple']
first_date = quotes_Apple.iloc[0]['date'][0:10]
Company_Apple =['Microsoft','IBM','Samsung','Dell','Apple'] 
ticker_Apple =[ticker_of_company('Microsoft'),
               ticker_of_company('IBM'),
               ticker_of_company('Samsung'),
               ticker_of_company('Dell'),
              ticker_of_company('Apple')] 
shares_Apple =[yf.Ticker(ticker_Apple[0]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[1]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[2]).info['sharesOutstanding'],
              yf.Ticker(ticker_Apple[3]).info['sharesOutstanding'],
               yf.Ticker(ticker_Apple[4]).info['sharesOutstanding']]
hist_Apple =[yf.Ticker(ticker_Apple[0]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[1]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[2]).history(start=first_date)[['Close', 'Volume']],
              yf.Ticker(ticker_Apple[3]).history(start=first_date)[['Close', 'Volume']],
               yf.Ticker(ticker_Apple[4]).history(start=first_date)[['Close', 'Volume']]]
after =time.time()
print('fetch time:',str(after-before))

In [ ]:
hist_Apple[0].iloc[0]

In [ ]:
hist_Apple[0].iloc[0].name

In [ ]:
hist_Apple[0].iloc[0].name.date() > datetime.date.fromisoformat('2015-12-04')

In [ ]:
for i in range(len(hist_Apple[0])):
    if hist_Apple[0].iloc[i].name.date() > datetime.date.fromisoformat('2015-12-12'):
        print(hist_Apple[0].iloc[i])
        break